In [9]:
from openai import OpenAI

client = OpenAI()

# assistant = client.beta.assistants.create(
#     name="Book Assistant",
#     instructions="You help users with their question on the files they upload.",
#     model="gpt-4o-mini",
#     tools=[{"type": "file_search"}],
# )
assistant_id = "asst_CfJuj6Ak8d4gAk1ZUtd9toWV"

In [13]:
vector_store = client.beta.vector_stores.create(name="Book Assistant")

file_paths = ["./files/chapter_one.txt"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)

completed


In [14]:
assistant = client.beta.assistants.update(
    assistant_id=assistant_id,
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store.id],
        }
    },
)

In [5]:
message_file = client.files.create(
    file=open("./files/chapter_one.txt", "rb"), purpose="assistants"
)

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want you to help me with this file",
            "attachments": [
                {
                    "file_id": message_file.id,
                    "tools": [
                        {
                            "type": "file_search",
                        }
                    ],
                }
            ],
        }
    ]
)
print(thread)

Thread(id='thread_HnAHUiYew3H81rczpDTKKp4H', created_at=1737799421, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [ ]:
# client.beta.threads.messages.create(
#     # message 내용 없이 파일만 전달
#     thread_id=thread.id,
#     role="user",
#     content="",
#     file_ids=[message_file.id],
# )

In [28]:
# 메시지를 주고 나면 항상 run을 해줘야 ai가 다음 작업을 수행함
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_xZ8j0fDh5lWWhZytyH6PUkVq', assistant_id='asst_CfJuj6Ak8d4gAk1ZUtd9toWV', cancelled_at=None, completed_at=None, created_at=1737801458, expires_at=1737802058, failed_at=None, incomplete_details=None, instructions='You help users with their question on the files they upload.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_HnAHUiYew3H81rczpDTKKp4H', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [3]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source: {annotation.file_citation}")


# def get_tool_outputs(run_id, thread_id):
#     run = get_run(run_id, thread_id)
#     outputs = []
#     for action in run.required_action.submit_tool_outputs.tool_calls:
#         action_id = action.id
#         function = action.function
#         print(f"Calling function: {function.name} with arg {function.arguments}")
#         outputs.append(
#             {
#                 "output": functions_map[function.name](json.loads(function.arguments)),
#                 "tool_call_id": action_id,
#             }
#         )
#     return outputs


# def submit_tool_outputs(run_id, thread_id):
#     outpus = get_tool_outputs(run_id, thread_id)
#     return client.beta.threads.runs.submit_tool_outputs(
#         run_id=run_id,
#         thread_id=thread_id,
#         tool_outputs=outpus,
#     )

In [29]:
get_run(run.id, thread.id).status

'in_progress'

In [19]:
get_messages(thread.id)

user: I want you to help me with this file
assistant: Sure! Please let me know what specific information or assistance you need with the file.


In [20]:
send_message(
    thread.id,
    "I want to know where does Winston live and how his apartment looks like.",
)

Message(id='msg_ZFlWSnsnHprru3CPNXI59UZ6', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know where does Winston live and how his apartment looks like.'), type='text')], created_at=1737801194, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_HnAHUiYew3H81rczpDTKKp4H')

In [25]:
get_messages(thread.id)

user: I want you to help me with this file
assistant: Sure! Please let me know what specific information or assistance you need with the file.
user: I want to know where does Winston live and how his apartment looks like.
assistant: Winston Smith lives in an apartment located in a building called Victory Mansions. His living environment is quite bleak and reflects the dismal state of society in the story. Here are some key points about his living situation and apartment:

1. **Location and Condition**: 
   - Winston's apartment is described as part of Victory Mansions, and the building appears to be in a poor state, as it is associated with the oppressive regime of the Party. The hallway smells of boiled cabbage and old rag mats, contributing to the grim atmosphere【6:0†source】.

2. **Climbing Stairs**: 
   - His flat is located seven flights up, indicating a significant amount of stairs to climb, especially since the lift is often out of order or not functioning【6:0†source】. This detai

In [26]:
send_message(
    thread.id,
    "Where does he wokr?",
)

Message(id='msg_3s9Ceu44A2cwCZuyig0GR51f', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Where does he wokr?'), type='text')], created_at=1737801432, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_HnAHUiYew3H81rczpDTKKp4H')

In [30]:
get_run(run.id, thread.id).status

'completed'

In [31]:
get_messages(thread.id)

user: I want you to help me with this file
assistant: Sure! Please let me know what specific information or assistance you need with the file.
user: I want to know where does Winston live and how his apartment looks like.
assistant: Winston Smith lives in an apartment located in a building called Victory Mansions. His living environment is quite bleak and reflects the dismal state of society in the story. Here are some key points about his living situation and apartment:

1. **Location and Condition**: 
   - Winston's apartment is described as part of Victory Mansions, and the building appears to be in a poor state, as it is associated with the oppressive regime of the Party. The hallway smells of boiled cabbage and old rag mats, contributing to the grim atmosphere【6:0†source】.

2. **Climbing Stairs**: 
   - His flat is located seven flights up, indicating a significant amount of stairs to climb, especially since the lift is often out of order or not functioning【6:0†source】. This detai